![OpenSARlab notebook banner](NotebookAddons/blackboard-banner.png)

# Change Point Detection in SAR Amplitude Time Series Data

### Franz J Meyer; University of Alaska Fairbanks & Josef Kellndorfer, [Earth Big Data, LLC](http://earthbigdata.com/)
<img src="NotebookAddons/UAFLogo_A_647.png" width="170" align="right" />


This notebook applies Change Point Detection on a deep multi-temporal SAR image data stack acquired by Sentinel-1. Specifically, the lab applies the method of *Cumulative Sums* to perform change detection on a 60 image deep Sentinel-1 data stack over Niamey, Niger.  

**In this notebook we introduce the following data analysis concepts:**

- How to use your own HyP3-generated data stack in a change detection effort
- The concepts of time series slicing by month, year, and date.
- The concepts and workflow of Cumulative Sum-based change point detection.
- The identification of change dates for each identified change point.

---
**Important Note about JupyterHub**

Your JupyterHub server will automatically shutdown when left idle for more than 1 hour. Your notebooks will not be lost but you will have to restart their kernels and re-run them from the beginning. You will not be able to seamlessly continue running a partially run notebook.


In [ ]:

%%javascript
var kernel = Jupyter.notebook.kernel;
var command = ["notebookUrl = ",
               "'", window.location, "'" ].join('')
kernel.execute(command)

In [ ]:
from IPython.display import Markdown
from IPython.display import display

user = !echo $JUPYTERHUB_USER
env = !echo $CONDA_PREFIX
if env[0] == '':
    env[0] = 'Python 3 (base)'
if env[0] != '/home/jovyan/.local/envs/rtc_analysis':
    display(Markdown(f'<text style=color:red><strong>WARNING:</strong></text>'))
    display(Markdown(f'<text style=color:red>This notebook should be run using the "rtc_analysis" conda environment.</text>'))
    display(Markdown(f'<text style=color:red>It is currently using the "{env[0].split("/")[-1]}" environment.</text>'))
    display(Markdown(f'<text style=color:red>Select the "rtc_analysis" from the "Change Kernel" submenu of the "Kernel" menu.</text>'))
    display(Markdown(f'<text style=color:red>If the "rtc_analysis" environment is not present, use <a href="{notebookUrl.split("/user")[0]}/user/{user[0]}/notebooks/conda_environments/Create_OSL_Conda_Environments.ipynb"> Create_OSL_Conda_Environments.ipynb </a> to create it.</text>'))
    display(Markdown(f'<text style=color:red>Note that you must restart your server after creating a new environment before it is usable by notebooks.</text>'))

## 0. Importing Relevant Python Packages

Our first step is to **import the necessary python libraries into your Jupyter Notebook:**

In [ ]:
%%capture
from pathlib import Path
import json # for loads

import pandas as pd
from osgeo import gdal
import numpy as np

from ipyfilechooser import FileChooser

%matplotlib inline 
import matplotlib.pylab as plt

import opensarlab_lib as asfn
asfn.jupytertheme_matplotlib_format()

## 1. Load Your Prepared Data Stack Into the Notebook

This notebook assumes that you've prepared your own data stack of **RTC image products** over your personal area of interest. This can be done using the **Prepare_Data_Stack_Hyp3_v2** and **Subset_Data_Stack notebooks**.
    
This notebook expects [Radiometric Terrain Corrected](https://media.asf.alaska.edu/uploads/RTC/rtc_atbd_v1.2_final.pdf) (RTC) image products as input, so be sure to select an RTC process when creating the subscription for your input data within HyP3. Prefer a **unique orbit geometry** (ascending or descending) to keep geometric differences between images low. 

**Begin by writing a function to retrieve and the absolute paths to each of our tiffs:**

In [ ]:
def get_tiff_paths(paths):
    tiff_paths = !ls $paths | sort -t_ -k5,5
    return tiff_paths

**Select the directory holding your tiffs**
- Click the `Select` button
- Navigate to your data directory
- Click the `Select` button
- Confirm that the desired path appears in green text
- Click the `Change` button to alter your selection

In [ ]:
fc = FileChooser('/home/jovyan/notebooks')
display(fc)

**Determine the path to the analysis directory containing the tiff directory:**

In [ ]:
tiff_dir = Path(fc.selected_path)
analysis_dir = tiff_dir.parent
print(f"analysis_dir: {analysis_dir}")

paths = tiff_dir/"*.tif*"
tiff_paths = get_tiff_paths(paths)

**Create a wildcard path to the tiffs:**

In [ ]:
wildcard_path = f"{tiff_dir}/*.tif*"
print(wildcard_path)

**Write a function to extract the tiff dates from a wildcard path:**

In [ ]:
def get_dates(paths):
    dates = []
    pths = list(tiff_dir.rglob('*.tif*'))

    for p in pths:
        filename = p.name.split('_')
    
        for chunk in filename:
            if len(chunk) == 15 and 'T' in chunk:
                date = chunk.split('T')[0]
                dates.append(date)
                break
            elif len(chunk) == 8:
                try:
                    int(chunk)
                    dates.append(chunk)
                    break
                except ValueError:
                    continue              
    dates.sort()
    return dates

**Call get_dates() to collect the product acquisition dates:**

In [ ]:
dates = get_dates(tiff_dir)
print(dates)

**Gather the upper-left and lower-right corner coordinates of the data stack:**

In [ ]:
coords = [[], []]
info = (gdal.Info(tiff_paths[0], options = ['-json']))
info = json.dumps(info)
coords[0] = (json.loads(info))['cornerCoordinates']['upperLeft']
coords[1] = (json.loads(info))['cornerCoordinates']['lowerRight']
print(coords)

**Grab the stack's UTM zone.**

Note that any UTM zone conflicts should already have been handled in the Prepare_Data_Stack_Hyp3 notebook.

In [ ]:
utm = json.loads(info)['coordinateSystem']['wkt'].split('ID')[-1].split(',')[1][0:-2]
print(f"UTM Zone: {utm}")

---
Now we stack up the data by creating a virtual raster table with links to all subset data files.

**Create the virtual raster table for the subset GeoTiffs:**

In [ ]:
image_file = Path(f"{analysis_dir}/raster_stack.vrt")
!gdalbuildvrt -separate $image_file $wildcard_path

---
## 3. Now You Can Work With Your Data

Now you are ready to perform time series change detection on your data stack.

### 3.1 Create an index of timedelta64 data with Pandas

In [ ]:
# Get some indices for plotting
time_index = pd.DatetimeIndex(dates)

**Print the bands and dates for all images in the virtual raster table (VRT):**

In [ ]:
j = 1
print(f"Bands and dates for {image_file}")
for i in time_index:
    print("{:4d} {}".format(j, i.date()), end=' ')
    j += 1
    if j%5 == 1: print()

--- 
### 3.2 Open Your Data Stack with gdal

In [ ]:
img = gdal.Open(str(image_file))

**Print the bands, pixels, and lines:**

In [ ]:
print(f"Number of  bands: {img.RasterCount}")
print(f"Number of pixels: {img.RasterXSize}")
print(f"Number of  lines: {img.RasterYSize}")

---
### 3.3 Create a masked raster stack

In [ ]:
raster_stack = img.ReadAsArray()
raster_stack_masked = np.ma.masked_where(raster_stack==0, raster_stack)
del raster_stack

---
## 4. Cumulative Sum-based Change Detection Across an Entire Image

Using numpy arrays we can apply the concept of **cumulative sum change detection** analysis effectively on the entire image stack. We take advantage of array slicing and axis-based computing in numpy. **Axis 0 is the time domain** in our raster stacks.
    
---
### 4.1 Create our time series stack

**Calculate the dB scale:**

In [ ]:
db = 10.*np.ma.log10(raster_stack_masked)

Sometimes it makes sense to **extract a reduced time span** from the full time series to reduce the number of different change objects in a scene. In the following, we extract a shorter time span:

In [ ]:
date_picker = asfn.gui_date_picker(dates)
date_picker

In [ ]:
subset_dates = date_picker.value
subset_dates = pd.DatetimeIndex(subset_dates)
date_index_subset = np.where((time_index>=subset_dates[0]) & (time_index<=subset_dates[1]))
db_subset = np.squeeze(db[date_index_subset, :, :])
time_index_subset = time_index[date_index_subset]

In [ ]:
plt.figure(figsize=(12, 8))
band_number = 0
vmin = np.percentile(db_subset[band_number], 5)
vmax = np.percentile(db_subset[band_number], 95)
plt.title('Band  {} {}'.format(band_number+1, time_index_subset[band_number].date()))
plt.imshow(db_subset[0], cmap='gray', vmin=vmin, vmax=vmax)
cbar = plt.colorbar()
_ = cbar.ax.set_xlabel('dB', fontsize='12')

---
### 4.2 Calculate Mean Across Time Series to Prepare for Calculation of Cummulative Sum $S$:

**Write a function to convert our plots into GeoTiffs:**

In [ ]:
def geotiff_from_plot(source_image, out_filename, extent, utm, cmap=None, vmin=None, vmax=None, interpolation=None, dpi=300):
    assert "." not in out_filename, 'Error: Do not include the file extension in out_filename'
    assert type(extent) == list and len(extent) == 2 and len(extent[0]) == 2 and len(
        extent[1]) == 2, 'Error: extent must be a list in the form [[upper_left_x, upper_left_y], [lower_right_x, lower_right_y]]'
    
    plt.figure()
    plt.axis('off')
    plt.imshow(source_image, cmap=cmap, vmin=vmin, vmax=vmax, interpolation=interpolation)
    temp = Path(f"{out_filename}_temp.png")
    plt.savefig(temp, dpi=dpi, transparent='true', bbox_inches='tight', pad_inches=0)

    cmd = f"gdal_translate -of Gtiff -a_ullr {extent[0][0]} {extent[0][1]} {extent[1][0]} {extent[1][1]} -a_srs EPSG:{utm} {temp} {out_filename}.tiff"
    !{cmd}
    try:
        temp.unlink()
    except FileNotFoundError:
        pass

**Create a directory in which to store our plots and animations:**

In [ ]:
output_path = analysis_dir/'plots_and_animations'

if not output_path.exists():
    output_path.mkdir()

**Plot the time-series mean and save as a png (time_series_mean.png):**

In [ ]:
db_mean = np.mean(db_subset, axis=0)
plt.figure(figsize=(12, 8))
plt.imshow(db_mean, cmap='gray')
cbar = plt.colorbar()
cbar.ax.set_xlabel('dB', fontsize='12')
plt.savefig(f"{output_path}/time_series_mean.png", dpi=300, transparent='true')

**Save the time-series mean as a GeoTiff (time_series_mean.tiff):**

In [ ]:
%%capture
geotiff_from_plot(db_mean, f"{output_path}/time_series_mean", coords, utm, cmap='gray')

**Calculate the residuals and plot residuals\[0\]. Save it as a png (residuals.png):**

In [ ]:
residuals = db_subset - db_mean

plt.figure(figsize=(12, 8))
plt.imshow(residuals[0])
plt.title('Residuals for Band  {} {}'.format(band_number+1, time_index_subset[band_number].date()))
cbar = plt.colorbar()
_ = cbar.ax.set_xlabel('dB', fontsize='12')
plt.savefig(f"{output_path}/residuals.png", dpi=300, transparent='true')

**Save the residuals\[0\] as a GeoTiff (residuals.tiff):**

In [ ]:
%%capture
geotiff_from_plot(residuals[0], f"{output_path}/residuals", coords, utm)

---
### 4.3 Calculate Cummulative Sum $S$ as well as Change Magnitude $S_{diff}$:

**Plot Smin, Smax, and the change magnitude and save a png of the plots (Smin_Smax_Sdiff.png):**

In [ ]:
summation = np.cumsum(residuals, axis=0)
summation_max = np.max(summation, axis=0)
summation_min = np.min(summation, axis=0)
change_mag = summation_max - summation_min
fig, ax = plt.subplots(1, 3, figsize=(16, 4))
vmin = np.percentile(summation_min.flatten(), 3)
vmax = np.percentile(summation_max.flatten(), 97)
max_plot = ax[0].imshow(summation_max, vmin=vmin, vmax=vmax)
ax[0].set_title('$S_{max}$')
ax[1].imshow(summation_min, vmin=vmin, vmax=vmax)
ax[1].set_title('$S_{min}$')
ax[2].imshow(change_mag, vmin=vmin, vmax=vmax)
ax[2].set_title('Change Magnitude')
fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.85, 0.15, 0.02, 0.7])
cbar = fig.colorbar(max_plot, cax=cbar_ax)
_ = cbar.ax.set_xlabel('dB', fontsize='12')
plt.savefig(f"{output_path}/Smin_Smax_Sdiff.png", dpi=300, transparent='true')

**Save Smax as a GeoTiff (Smax.tiff):**

In [ ]:
%%capture
geotiff_from_plot(summation_max, f"{output_path}/Smax", coords, utm, vmin=vmin, vmax=vmax)

**Save Smin as a GeoTiff (Smin.tiff):**

In [ ]:
%%capture
geotiff_from_plot(summation_min, f"{output_path}/Smin", coords, utm, vmin=vmin, vmax=vmax)

**Save the change magnitude as a GeoTiff (Sdiff.tiff):**

In [ ]:
%%capture
geotiff_from_plot(change_mag, f"{output_path}/Sdiff", coords, utm, vmin=vmin, vmax=vmax)

---
### 4.4 Mask $S_{diff}$ With a-priori Threshold To Idenfity Change Candidates:

To identified change candidate pixels, we can threshold $S_{diff}$ to reduce computation of the bootstrapping. For land cover change, we would not expect more than 5-10% change pixels in a landscape. So, if the test region is reasonably large, setting a threshold for expected change to 10% is appropriate. In our example, we'll start out with a very conservative threshold of 50%.

**Plot and tsave the histogram and CDF for the change magnitude (change_mag_histogram_CDF.png):**

In [ ]:
plt.rcParams.update({'font.size': 14})
fig = plt.figure(figsize=(14, 6)) # Initialize figure with a size
ax1 = fig.add_subplot(121)  # 121 determines: 2 rows, 2 plots, first plot
ax2 = fig.add_subplot(122)
# Second plot: Histogram
# IMPORTANT: To get a histogram, we first need to *flatten* 
# the two-dimensional image into a one-dimensional vector.
histogram = ax1.hist(change_mag.flatten(), bins=200, range=(0, np.max(change_mag)))
ax1.xaxis.set_label_text('Change Magnitude')
ax1.set_title('Change Magnitude Histogram')
plt.grid()
n, bins, patches = ax2.hist(change_mag.flatten(), bins=200, range=(0, np.max(change_mag)), cumulative='True', density='True', histtype='step', label='Empirical')
ax2.xaxis.set_label_text('Change Magnitude')
ax2.set_title('Change Magnitude CDF')
plt.grid()
plt.savefig(f"{output_path}/change_mag_histogram_CDF", dpi=72)

In [ ]:
precentile = 0.5
out_indicies = np.where(n>precentile)
threshold_index = np.min(out_indicies)
threshold = bins[threshold_index]
print('At the {}% percentile, the threshold value is {:2.2f}'.format(precentile*100, threshold))

Using this threshold, we can **visualize our change candidate areas and save them as a png (change_candidate.png):**

In [ ]:
change_mag_mask = change_mag < threshold
plt.figure(figsize=(12, 8))
plt.title('Change Candidate Areas (black)')
_ = plt.imshow(change_mag_mask, cmap='gray')
plt.savefig(f"{output_path}/change_candidate.png", dpi=300, transparent='true')

**Save the change candidate areas as a GeoTiff (change_canididate.tiff):**

In [ ]:
%%capture
geotiff_from_plot(change_mag_mask, f"{output_path}/change_canididate", coords, utm, cmap='gray')

---
### 4.5 Bootstrapping to Prepare for Change Point Selection:

We can now perform bootstrapping over the candidate pixels. The workflow is as follows:

- Filter our residuals to the change candidate pixels
- Perform bootstrapping over candidate pixels

For efficient computing we permutate the index of the time axis.

In [ ]:
residuals_mask = np.broadcast_to(change_mag_mask , residuals.shape)
residuals_masked = np.ma.array(residuals, mask=residuals_mask)

**On the masked time series stack of residuals, we can re-compute the cumulative sums:**

In [ ]:
summation_masked = np.ma.cumsum(residuals_masked, axis=0)

**Plot the masked Smax, Smin, and change magnitude. Save them as a png (masked_Smax_Smin_Sdiff.png):**

In [ ]:
summation_masked_max = np.ma.max(summation_masked, axis=0)
summation_masked_min = np.ma.min(summation_masked, axis=0)
change_mag_masked = summation_masked_max - summation_masked_min
fig, ax = plt.subplots(1, 3, figsize=(16, 4))
vmin = summation_masked_min.min()
vmax = summation_masked_max.max()
masked_sum_max_plot = ax[0].imshow(summation_masked_max, vmin=vmin, vmax=vmax)
ax[0].set_title('Masked $S_{max}$')
ax[1].imshow(summation_masked_min, vmin=vmin, vmax=vmax)
ax[1].set_title('Masked $S_{min}$')
ax[2].imshow(change_mag_masked, vmin=vmin, vmax=vmax)
ax[2].set_title('Masked Change Magnitude')
fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.85, 0.15, 0.02, 0.7])
cbar = fig.colorbar(masked_sum_max_plot, cax=cbar_ax)
_ = cbar.ax.set_xlabel('dB', fontsize='12')
plt.savefig(f"{output_path}/masked_Smax_Smin_Sdiff.png", dpi=300, transparent='true')

**Save the masked Smax as a GeoTiff (masked_Smax.tiff):**

In [ ]:
%%capture
geotiff_from_plot(summation_masked_max, f"{output_path}/masked_Smax", coords, utm, vmin=vmin, vmax=vmax)

**Save the masked Smin as a GeoTiff (masked_Smin.tiff):**

In [ ]:
%%capture
geotiff_from_plot(summation_masked_min, f"{output_path}/masked_Smin", coords, utm, vmin=vmin, vmax=vmax)

**Save the masked change magnitude as a GeoTiff (masked_Sdiff.tiff):**

In [ ]:
%%capture
geotiff_from_plot(change_mag_masked, f"{output_path}/masked_Sdiff", coords, utm, vmin=vmin, vmax=vmax)

**Now let's perform bootstrapping:**

In [ ]:
random_index = np.random.permutation(residuals_masked.shape[0])
residuals_random = residuals_masked[random_index,:,:]

In [ ]:
n_bootstraps = 100  # bootstrap sample size

# to keep track of the maxium Sdiff of the bootstrapped sample:
change_mag_random_max = np.ma.copy(change_mag_masked) 
change_mag_random_max[~change_mag_random_max.mask]=0
# to compute the Sdiff sums of the bootstrapped sample:
change_mag_random_sum = np.ma.copy(change_mag_masked) 
change_mag_random_sum[~change_mag_random_max.mask]=0
# to keep track of the count of the bootstrapped sample
n_change_mag_gt_change_mag_random = np.ma.copy(change_mag_masked) 
n_change_mag_gt_change_mag_random[~n_change_mag_gt_change_mag_random.mask]=0
print("Running Bootstrapping for %4.1f iterations ..." % (n_bootstraps))
for i in range(n_bootstraps):
    # For efficiency, we shuffle the time axis index and use that 
    #to randomize the masked array
    random_index = np.random.permutation(residuals_masked.shape[0])
    # Randomize the time step of the residuals
    residuals_random = residuals_masked[random_index,:,:]  
    summation_random = np.ma.cumsum(residuals_random, axis=0)
    summation_random_max = np.ma.max(summation_random, axis=0)
    summation_random_min = np.ma.min(summation_random, axis=0)
    change_mag_random = summation_random_max - summation_random_min
    change_mag_random_sum += change_mag_random
    change_mag_random_max[np.ma.greater(change_mag_random, change_mag_random_max)] = \
    change_mag_random[np.ma.greater(change_mag_random, change_mag_random_max)]
    n_change_mag_gt_change_mag_random[np.ma.greater(change_mag_masked, change_mag_random)] += 1
    if ((i+1)/n_bootstraps*100)%10 == 0:
        print("\r%4.1f%% completed" % ((i+1)/n_bootstraps*100), end='\r', flush=True)
print(f"Bootstrapping Complete")

---
### 4.6 Extract Confidence Metrics and Select Final Change Points:

**We first compute for all pixels the confidence level $CL$, the change point significance metric $CP_{significance}$ and the product of the two as our confidence metric for identified change points. Plot the results and save them as a png (confidenceLevel_CPSignificance.png):**

In [ ]:
confidence_level = n_change_mag_gt_change_mag_random / n_bootstraps
change_point_significance = 1.- (change_mag_random_sum / n_bootstraps)/change_mag 
#Plot
fig, ax = plt.subplots(1, 3, figsize=(16, 4))
a = ax[0].imshow(confidence_level*100)
cbar0 = fig.colorbar(a, ax=ax[0])
_ = cbar0.ax.set_xlabel('%', fontsize='12')
ax[0].set_title('Confidence Level %')
a = ax[1].imshow(change_point_significance)
_ = fig.colorbar(a, ax=ax[1])
ax[1].set_title('Significance')
a = ax[2].imshow(confidence_level*change_point_significance)
_ = fig.colorbar(a, ax=ax[2])
_ = ax[2].set_title('CL x S')
plt.savefig(f"{output_path}/confidenceLevel_CPSignificance.png", dpi=300, transparent='true')

**Save the confidence level as a GeoTiff (confidence_level.tiff):**

In [ ]:
%%capture
geotiff_from_plot(confidence_level*100, f"{output_path}/confidence_level", coords, utm)

**Save the change point significance as a GeoTiff (cp_significance.tiff):**

In [ ]:
%%capture
geotiff_from_plot(change_point_significance, f"{output_path}/cp_significance", coords, utm)

**Save the change point significance as a GeoTiff (cp_significance.tiff):**

In [ ]:
%%capture
geotiff_from_plot(confidence_level*change_point_significance, f"{output_path}/confidenceLevel_x_CPSignificance", coords, utm)

**Now we can set a change point threshold to identify most likely change pixels in our map of change candidates:**

In [ ]:
change_point_threshold = 0.01

**Plot the detected change pixels based on the change_point_threshold and save it as a png (detected_change_pixels.png):**

In [ ]:
fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(1, 1, 1)
plt.title('Detected Change Pixels based on Threshold %2.2f' % (change_point_threshold))
a = ax.imshow(confidence_level*change_point_significance < change_point_threshold, cmap='cool')
plt.savefig(f"{output_path}/detected_change_pixels.png", dpi=300, transparent='true')

**Save the detected_change_pixels as a GeoTiff (detected_change_pixels.tiff):**

In [ ]:
%%capture
geotiff_from_plot(confidence_level*change_point_significance < change_point_threshold, f"{output_path}/detected_change_pixels", coords, utm, cmap='cool')

--- 
### 4.7 Derive Timing of Change for Each Change Pixel:

Our last step in the identification of the change points is to extract the timing of the change. We will produce a raster layer that shows the band number of this first date after a change was detected. We will make use of the numpy indexing scheme. First, we create a combined mask of the first threshold and the identified change points after the bootstrapping. For this we use the numpy "mask_or" operation.

In [ ]:
# make a mask of our change points from the new threhold and the previous mask
change_point_mask = np.ma.mask_or(confidence_level*change_point_significance < change_point_threshold, confidence_level.mask)
# Broadcast the mask to the shape of the masked S curves
change_point_mask2 = np.broadcast_to(change_point_mask, summation_masked.shape)
# Make a numpy masked array with this mask
change_point_raster = np.ma.array(summation_masked.data, mask=change_point_mask2)

To retrieve the dates of the change points we find the band indices in the time series along the time axis where the maximum of the cumulative sums was located. Numpy offers the "argmax" function for this purpose.

In [ ]:
change_point_index = np.ma.argmax(change_point_raster, axis=0)
change_indices = list(np.unique(change_point_index))
print(change_indices)
change_indices.remove(0)
print(change_indices)
# Look up the dates from the indices to get the change dates
all_dates = time_index_subset
change_dates = [str(all_dates[x].date()) for x in change_indices]

**Lastly, we plot the change dates by showing the $CP_{index}$ raster and label the change dates. Save the plot as a png (change_dates.png):**

In [ ]:
ticks = change_indices
ticklabels = change_dates

cmap = plt.cm.get_cmap('tab20', ticks[-1])
fig, ax = plt.subplots(figsize=(12, 12))
cax = ax.imshow(change_point_index, interpolation='nearest', cmap=cmap)
# fig.subplots_adjust(right=0.8)
# cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
# fig.colorbar(p,cax=cbar_ax)

ax.set_title('Dates of Change')
# cbar = fig.colorbar(cax,ticks=ticks)
cbar = fig.colorbar(cax, ticks=ticks, orientation='horizontal')
_ = cbar.ax.set_xticklabels(ticklabels, size=10, rotation=45, ha='right')
plt.savefig(f"{output_path}/change_dates.png", dpi=300, transparent='true')

**Save the change dates as a GeoTiff (change_dates.tiff):**

In [ ]:
%%capture
geotiff_from_plot(change_point_index, f"{output_path}/change_dates", coords, utm, cmap=cmap, interpolation='nearest', dpi=600)

*GEOS 657 Microwave Remote Sensing - Version 1.4.1 - November 2021*
 
*Version Changes*

- asf_notebook -> opensarlab_lib
- html -> markdown